In [14]:
import os
import json
from utils import save_jsonl, read_jsonl

# 🔮 Get Wikidata KB

In [15]:
import requests

def fetch_labels(entity_ids):
    """
    Fetch labels for a given list of Wikidata entity IDs.

    Parameters:
    - entity_ids: List of Wikidata entity IDs.

    Returns:
    - A dictionary mapping each Wikidata entity ID to its label.
    """
    labels = {}
    # Wikidata API may have limits on the number of IDs per request; adjust if necessary.
    for i in range(0, len(entity_ids), 50):
        batch_ids = entity_ids[i:i+50]
        params = {
            'action': 'wbgetentities',
            'ids': '|'.join(batch_ids),
            'props': 'labels',
            'languages': 'en',
            'format': 'json'
        }
        response = requests.get("https://www.wikidata.org/w/api.php", params=params)
        response_json = response.json()

        for entity_id, entity in response_json['entities'].items():
            label = entity.get('labels', {}).get('en', {}).get('value', entity_id)
            labels[entity_id] = label
    return labels

def format_value(value, labels):
    """
    Convert a Wikidata value to a string representation, using labels for entity IDs.
    """
    if isinstance(value, dict):
        # For entity IDs, replace with label if available
        if 'id' in value:
            return labels.get(value['id'], value['id'])
        elif 'time' in value:
            return value['time']
        elif 'amount' in value:
            return value['amount']
        else:
            return str(value)
    else:
        return str(value)

def get_all_properties_with_labels(wikidata_id):
    """
    Fetch all properties and their values for a given Wikidata ID, 
    including labels for properties and their entity values, encapsulated by the entity name.

    Parameters:
    - wikidata_id: The Wikidata ID of the entity.

    Returns:
    - A dictionary with the entity label as the key and the properties dictionary as the value.
    """
    # Initial API call to get all claims/properties for the entity
    params = {
        'action': 'wbgetentities',
        'ids': wikidata_id,
        'props': 'claims|labels',
        'languages': 'en',
        'format': 'json'
    }
    response = requests.get("https://www.wikidata.org/w/api.php", params=params)
    entity_data = response.json()['entities'][wikidata_id]

    # Fetch the label for the entity
    entity_label = entity_data.get('labels', {}).get('en', {}).get('value', wikidata_id)

    claims = entity_data['claims']

    # Collect property IDs and value entity IDs for label fetching
    prop_ids = list(claims.keys())
    value_entity_ids = set()
    for prop_id in prop_ids:
        for claim in claims[prop_id]:
            if 'datavalue' in claim['mainsnak']:
                data_value = claim['mainsnak']['datavalue'].get('value')
                if isinstance(data_value, dict) and 'id' in data_value:
                    value_entity_ids.add(data_value['id'])

    # Fetch labels for all property IDs and value entity IDs
    all_labels = fetch_labels(list(set(prop_ids) | value_entity_ids))

    # Construct the result dictionary with labels
    properties_result = {}
    for prop_id in prop_ids:
        prop_label = all_labels.get(prop_id, prop_id)
        properties_result[prop_label] = []
        for claim in claims[prop_id]:
            if 'datavalue' in claim['mainsnak']:
                data_value = claim['mainsnak']['datavalue'].get('value')
                formatted_value = format_value(data_value, all_labels)
                properties_result[prop_label].append(formatted_value)

    # Encapsulate the result using the entity label
    encapsulated_result = {
        "entity_label": entity_label,
        "properties" : properties_result
        }

    return encapsulated_result

# Example usage
wikidata_id = 'Q155979'
props_with_labels = get_all_properties_with_labels(wikidata_id)

print(json.dumps(props_with_labels, indent=4, ensure_ascii=False))

{
    "entity_label": "Alexei Navalny",
    "properties": {
        "GND ID": [
            "1024799212"
        ],
        "member of": [
            "Russian Opposition Coordination Council",
            "Yale World Fellows"
        ],
        "sex or gender": [
            "male"
        ],
        "Library of Congress authority ID": [
            "n2012030899"
        ],
        "VIAF ID": [
            "254613339",
            "304693850"
        ],
        "educated at": [
            "Finance University under the Government of the Russian Federation",
            "Yale University",
            "Peoples' Friendship University of Russia",
            "Yale World Fellows"
        ],
        "image": [
            "Alexey Navalny (cropped) 1.jpg"
        ],
        "member of political party": [
            "Yabloko",
            "Progress Party",
            "Russia of the Future"
        ],
        "Commons category": [
            "Alexey Navalny"
        ],
        "Freebase ID"

# 📗Get unstructured Text (wikipedia page)

In [16]:
import requests

def fetch_wikipedia_page_content(wikidata_id):
    """
    Fetch the entire content of a Wikipedia page for a given Wikidata ID.

    Parameters:
    - wikidata_id: The Wikidata ID of the entity.

    Returns:
    - The content of the Wikipedia page as a string.
    """
    # Fetch the Wikipedia page title for the Wikidata ID
    params_wikidata = {
        'action': 'wbgetentities',
        'ids': wikidata_id,
        'props': 'sitelinks',
        'sitefilter': 'enwiki',
        'format': 'json'
    }
    response_wikidata = requests.get("https://www.wikidata.org/w/api.php", params=params_wikidata)
    response_json_wikidata = response_wikidata.json()
    sitelinks = response_json_wikidata['entities'][wikidata_id].get('sitelinks', {})
    enwiki_title = sitelinks.get('enwiki', {}).get('title', '')
    
    if not enwiki_title:
        return "Wikipedia page title not found for the given Wikidata ID."
    
    # Fetch the content of the Wikipedia page using the title
    params_wikipedia = {
        'action': 'query',
        'format': 'json',
        'titles': enwiki_title,
        'prop': 'extracts',
        'explaintext': True,  # Return plain text content for the entire page
    }
    response_wikipedia = requests.get("https://en.wikipedia.org/w/api.php", params=params_wikipedia)
    response_json_wikipedia = response_wikipedia.json()
    page = next(iter(response_json_wikipedia['query']['pages'].values()))
    content = page.get('extract', '')
    
    return content


In [17]:
# Example usage
wikidata_id = 'Q155979' 
wikipedia_content = fetch_wikipedia_page_content(wikidata_id)
print(wikipedia_content)

Alexei Anatolyevich Navalny (Russian: Алексей Анатольевич Навальный, IPA: [ɐlʲɪkˈsʲej ɐnɐˈtolʲjɪvʲɪtɕ nɐˈvalʲnɨj]; 4 June 1976 – 16 February 2024) was a Russian opposition leader, lawyer, anti-corruption activist, and political prisoner. He organised anti-government demonstrations and ran for office to advocate reforms against corruption in Russia and against President Vladimir Putin and his government. Navalny was founder of the Anti-Corruption Foundation (FBK). He was recognised by Amnesty International as a prisoner of conscience, and was awarded the Sakharov Prize for his work on human rights.
Through his social media channels, Navalny and his team published material about corruption in Russia, organised political demonstrations and promoted his campaigns. In a 2011 radio interview, he described Russia's ruling party, United Russia, as a "party of crooks and thieves", which became a popular epithet. Navalny and the FBK have published investigations detailing alleged corruption by h

# 🍔 Chunk long texts

rough maths -> 4 chars ~= 1 token.

16385 max token length for gpt-3.5.

PLUS we need room for prompt.

In [18]:
def chunk_text_recursively(text, max_chunk_size=15000, chunks=None):
    """
    Recursively splits the text into chunks of a maximum size, without splitting words.

    # Example usage
    text = "Your very long text goes here. Repeat this sentence to make the text longer." * 500  # Example text
    chunks = chunk_text_recursively(text)
    # Optionally, print the number of chunks and the first chunk to verify
    print(f"Total chunks: {len(chunks)}")
    print(f"First chunk (showing first 100 characters): {chunks[0][:100]}")
    #

    Parameters:
    - text (str): The text to be chunked.
    - max_chunk_size (int): The maximum size of each chunk. Default is 15000.
    - chunks (List[str]): Used during recursion to accumulate chunks. Should not be set when calling externally.

    Returns:
    - List[str]: A list of text chunks.
    """
    if chunks is None:
        chunks = []
    
    # Base case: if the text is shorter than the max chunk size, just add it to the chunks list
    if len(text) <= max_chunk_size:
        chunks.append(text)
        return chunks

    # Recursive case: find the last space within the max chunk size and split the text there
    split_point = text.rfind(' ', 0, max_chunk_size)
    
    # If we couldn't find a space, we have a very long word, so just split at the max_chunk_size
    if split_point == -1:
        split_point = max_chunk_size
    
    # Add the left part of the text to the chunks list
    chunks.append(text[:split_point])
    
    # Recursively process the remaining text
    return chunk_text_recursively(text[split_point+1:], max_chunk_size, chunks)


In [19]:
num_tokens = len(wikipedia_content) / 4  # 4 chars ~= 1 token
# 16385 max length for gpt-3.5-0124

if num_tokens > 15_000:
    chunked_content = chunk_text_recursively(wikipedia_content)

# Get WikiData Entity IDs

In [20]:
import requests

# Base URL for the Wikidata Query Service
endpoint_url = "https://query.wikidata.org/sparql"

# SPARQL query template for fetching 10 entities of a specific type
query_template = """
SELECT ?entity ?entityLabel WHERE {{
  ?entity wdt:P31 wd:{}.
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
}}
LIMIT 10
"""

# Entity types to query: People (Q5), Places (Q17334923), Organizations (Q43229), Events (Q1656682)
entity_types = {
    "People": "Q5",
    "Places": "Q17334923",
    "Organizations": "Q43229",
    "Events": "Q1656682"
}

# Function to execute SPARQL query
def execute_query(query):
    try:
        response = requests.get(endpoint_url, params={'format': 'json', 'query': query})
        response.raise_for_status()  # Raise an error on bad status
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(e)
        return None

# Fetch and print 10 entities for each type
wikidata_ids = {}
for category, type_id in entity_types.items():
    query = query_template.format(type_id)
    results = execute_query(query)
    print(f"--- {category} ---")
    if results:
        for result in results["results"]["bindings"]:
            entity_id = result["entity"]["value"].split('/')[-1]  # Extract entity ID from URL
            print(f"{entity_id} - {result['entityLabel']['value']}")
            wikidata_ids[entity_id] = result['entityLabel']['value']
    else:
        print("No results found.")
    print("\n")


--- People ---
Q23 - George Washington
Q42 - Douglas Adams
Q76 - Barack Obama
Q80 - Tim Berners-Lee
Q91 - Abraham Lincoln
Q157 - François Hollande
Q181 - Jimmy Wales
Q185 - Larry Sanger
Q186 - Ken Jennings
Q192 - David Cameron


--- Places ---
P189 - location of discovery
Q316504 - Lager Heuberg
Q1187096 - underline
Q2560211 - Comme chez Soi
Q3267025 - antique shop
Q5171511 - Cornell Central Campus
Q10659195 - Q10659195
Q10676720 - Q10676720
Q10678560 - Q10678560
Q10758528 - Q10758528




--- Organizations ---
Q312 - Apple
Q1065 - United Nations
Q4096 - Vélo'v
Q4383 - International Federation of Football History & Statistics
Q5175 - Kalachuri
Q7809 - UNESCO
Q7983 - International Mathematical Olympiad
Q8024 - Council of Science Editors
Q8093 - Nintendo
Q8340 - Matte World Digital


--- Events ---
Q42369 - Cannes Film Festival
Q50805 - WrestleMania I
Q128067 - World Quizzing Championship
Q129683 - 18 November Torchlight procession
Q151610 - Running of the Nudes
Q160721 - Nuremberg Medical Trial
Q165874 - Ghadir Khumm
Q167888 - World Animal Day
Q172754 - world's fair
Q181817 - Saint Patrick's Day




# 😄 Run as a Batch + Save 

In [21]:
# Run batch and save

# Wikidata entities for people, organizations, and geopolitical entities (GPEs)
wikidata_ids['Q155979'] = 'Alexei Navalny'

wikidata_entities_out = []
from tqdm import tqdm
for wikidata_id in tqdm(wikidata_ids.keys()):
    entity = get_all_properties_with_labels(wikidata_id)
    wikipedia_content = fetch_wikipedia_page_content(wikidata_id)

    num_tokens = len(wikipedia_content) / 4  # 4 chars ~= 1 token

    if num_tokens > 15_000:
        chunked_content = chunk_text_recursively(wikipedia_content)
    else:
        chunked_content = [wikipedia_content]
    
    entity['chunked_content'] = chunked_content

    entity['QID'] = wikidata_id

    wikidata_entities_out.append(entity)

save_jsonl(wikidata_entities_out, '../../data/wikidata_entities.jsonl')

  0%|          | 0/41 [00:00<?, ?it/s]

100%|██████████| 41/41 [01:29<00:00,  2.18s/it]

Saved to f'../../data/wikidata_entities.jsonl



# 👀 Make sure it looks ok

In [22]:
entities = read_jsonl('../../data/wikidata_entities.jsonl')
print(entities[0].keys())
print(entities[0]['properties'])

dict_keys(['entity_label', 'properties', 'chunked_content', 'QID'])
{'cause of death': ['epiglottitis', 'acute laryngitis'], 'RSL ID (person)': ['000025437'], 'place of death': ['Mount Vernon', 'Mount Vernon'], 'spouse': ['Martha Washington'], 'signature': ['George Washington signature.svg'], 'father': ['Augustine Washington'], 'mother': ['Mary Ball Washington'], 'country of citizenship': ['Kingdom of Great Britain', 'United States of America'], 'occupation': ['politician', 'military officer', 'farmer', 'cartographer', 'surveyor', 'engineer', 'statesperson', 'revolutionary', 'writer', 'army officer'], 'coat of arms': ['coat of arms of George Washington'], 'coat of arms image': ['Coat of Arms of George Washington.svg'], 'Commons category': ['George Washington'], 'image': ['Gilbert Stuart Williamstown Portrait of George Washington.jpg'], 'GND ID': ['11876439X'], 'Library of Congress authority ID': ['n86140996'], 'VIAF ID': ['31432428'], 'ISNI': ['0000000120958599'], 'IMDb ID': ['ch002822

# ݐ Get Property counts by Usage

Find the number of times each property has been used in Wikidata; tis will inform our filtering for evaluation later.

In [35]:
import pandas as pd

df = pd.read_html('https://www.wikidata.org/wiki/Wikidata:Database_reports/List_of_properties/all')[0]
print(len(df))

11641


In [36]:
# filter " ID " rows
df = df[~df['label'].str.contains(r'\bID\b')]
df = df[~df['label'].str.contains(r'\bid\b')]
print(len(df))
df

3889


,ID,label,description,Data type[1],Counts[2]
0,P6,head of government,"head of the executive power of this town, city...",WI,"42,788 M 74 N"
1,P10,video,"relevant video. For images, use the property P...",CM,"13,760 M 23 Q 2 N"
2,P14,traffic sign,"graphic symbol describing the item, used at th...",CM,"27,793 M 358 N"
3,P15,route map,image of route map at Wikimedia Commons,CM,"27,551 M 208 N"
4,P16,transport network,network the infrastructure is a part of,WI,"123,555 M 522 Q 32 N"
...,...,...,...,...,...
11627,P12439,civil rank,non-military rank of a civil office holder in ...,WI,3 M
11629,P12441,Anglo-Norman Dictionary entry,entry for a lexeme in the online Anglo-Norman ...,EI,3 M
11631,P12443,has cabinet,value is the advisory committee to the item's ...,WI,22 M
11634,P12446,ISCED field,mapping of a particular course or curriculum t...,WI,0


In [38]:
# see if there's any ID labels left
matching_rows = df[df['label'].str.contains('ID')]

# Display the matching rows
len(matching_rows)


40

In [39]:
import pandas as pd
import re

# Function to find the largest number in the string
def find_largest_number(s):
    # Find all number occurrences and remove commas
    numbers = re.findall(r'\d+', s.replace(',', ''))
    # Convert all found numbers to integers
    numbers = [int(num) for num in numbers]
    # Return the max number or NaN if there are no numbers
    return max(numbers) if numbers else float('nan')

# Apply the function to each entry in 'Counts[2]'
df['largest_number'] = df['Counts[2]'].apply(find_largest_number)
df['largest_number'] = pd.to_numeric(df['largest_number'], errors='coerce')
df_sorted = df.sort_values(by='largest_number', ascending=False)

df_sorted.head(3)


,ID,label,description,Data type[1],Counts[2],largest_number
2376,P2860,cites work,citation from one creative or scholarly work t...,WI,"292,583,247 M 390 N",292583247
1184,P1545,series ordinal,position of an item in its parent series (most...,S,"175,830,141 Q 2,298 N",175830141
1663,P2093,author name string,stores unspecified author or editor name for p...,S,"138,055,438 M 589,477 Q 29,173 R 213 N",138055438


In [27]:
df_sorted.to_csv('../../data/wikidata-properties-counts.csv', index=False, encoding='utf-8')

# Infoboxes

These might be useful.

EDIT: these probably aren't useful.

In [28]:
ent = entities[0]
import wptools
wiki_content = wptools.page('Alexei Navalny', wikibase='Q155979').get_parse()
infobox = wiki_content.data['infobox']
print(infobox)
ent['infobox'] = infobox

en.wikipedia.org (parse) Alexei Navalny


{'name': 'Alexei Navalny', 'native_name': '{{nobold|Алексей Навальный}}', 'native_name_lang': 'ru', 'image': 'Alexey Navalny 2 (cropped) 1.jpg', 'caption': 'Navalny in 2011', 'office': "Leader of [[Russia of the Future]] {{efn|Previously known as the People's Alliance (2012–2014) and the Progress Party (2014–2018)}}", 'term_start': '28 March 2019', 'term_end': '17 January 2021 {{efn|Arrested and subsequently imprisoned}}', 'deputy': '[[Leonid Volkov (politician)|Leonid Volkov]]', 'predecessor': '[[Ivan Zhdanov]]', 'successor': '[[Leonid Volkov (politician)|Leonid Volkov]] (acting)', 'term_start1': '17 November 2013', 'term_end1': '19 May 2018', 'predecessor1': 'Office established', 'successor1': '[[Ivan Zhdanov]]', 'office2': 'Chairman of the Session of the [[Russian Opposition Coordination Council]]', 'term_start2': '27 October 2012', 'term_end2': '24 November 2012', 'predecessor2': 'Office established', 'successor2': '[[Garry Kasparov]]', 'office3': 'Member of the [[Russian Oppositio

en.wikipedia.org (imageinfo) File:Alexey Navalny 2 (cropped) 1.jpg
Alexei Navalny (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Alexey Na...
  infobox: <dict(46)> name, native_name, native_name_lang, image, ...
  iwlinks: <list(12)> https://commons.wikimedia.org/wiki/Category:...
  pageid: 29174356
  parsetree: <str(384340)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Alexei Navalny
  wikibase: Q155979
  wikidata_url: https://www.wikidata.org/wiki/Q155979
  wikitext: <str(326955)> {{Short description|Russian opposition l...
}


In [29]:
for ent in entities:
    import wptools
    wiki_content = wptools.page('Alexei Navalny', wikibase='Q155979').get_parse()
    infobox = wiki_content.data['infobox']
    ent['infobox'] = infobox

en.wikipedia.org (parse) Alexei Navalny


en.wikipedia.org (imageinfo) File:Alexey Navalny 2 (cropped) 1.jpg
Alexei Navalny (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Alexey Na...
  infobox: <dict(46)> name, native_name, native_name_lang, image, ...
  iwlinks: <list(12)> https://commons.wikimedia.org/wiki/Category:...
  pageid: 29174356
  parsetree: <str(384340)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Alexei Navalny
  wikibase: Q155979
  wikidata_url: https://www.wikidata.org/wiki/Q155979
  wikitext: <str(326955)> {{Short description|Russian opposition l...
}
en.wikipedia.org (parse) Alexei Navalny
en.wikipedia.org (imageinfo) File:Alexey Navalny 2 (cropped) 1.jpg
Alexei Navalny (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Alexey Na...
  infobox: <dict(46)> name, native_name, native_name_lang, image, ...
  iwlinks: <list(12)> https://commons.wikimedia.org/wiki/Category:...
  pageid: 29174356
  parsetree: <str(384340)> <root><t

In [30]:
entities[0]['infobox']

{'name': 'Alexei Navalny',
 'native_name': '{{nobold|Алексей Навальный}}',
 'native_name_lang': 'ru',
 'image': 'Alexey Navalny 2 (cropped) 1.jpg',
 'caption': 'Navalny in 2011',
 'office': "Leader of [[Russia of the Future]] {{efn|Previously known as the People's Alliance (2012–2014) and the Progress Party (2014–2018)}}",
 'term_start': '28 March 2019',
 'term_end': '17 January 2021 {{efn|Arrested and subsequently imprisoned}}',
 'deputy': '[[Leonid Volkov (politician)|Leonid Volkov]]',
 'predecessor': '[[Ivan Zhdanov]]',
 'successor': '[[Leonid Volkov (politician)|Leonid Volkov]] (acting)',
 'term_start1': '17 November 2013',
 'term_end1': '19 May 2018',
 'predecessor1': 'Office established',
 'successor1': '[[Ivan Zhdanov]]',
 'office2': 'Chairman of the Session of the [[Russian Opposition Coordination Council]]',
 'term_start2': '27 October 2012',
 'term_end2': '24 November 2012',
 'predecessor2': 'Office established',
 'successor2': '[[Garry Kasparov]]',
 'office3': 'Member of the

In [31]:
entities[0]['properties']

{'cause of death': ['epiglottitis', 'acute laryngitis'],
 'RSL ID (person)': ['000025437'],
 'place of death': ['Mount Vernon', 'Mount Vernon'],
 'spouse': ['Martha Washington'],
 'signature': ['George Washington signature.svg'],
 'father': ['Augustine Washington'],
 'mother': ['Mary Ball Washington'],
 'country of citizenship': ['Kingdom of Great Britain',
  'United States of America'],
 'occupation': ['politician',
  'military officer',
  'farmer',
  'cartographer',
  'surveyor',
  'engineer',
  'statesperson',
  'revolutionary',
  'writer',
  'army officer'],
 'coat of arms': ['coat of arms of George Washington'],
 'coat of arms image': ['Coat of Arms of George Washington.svg'],
 'Commons category': ['George Washington'],
 'image': ['Gilbert Stuart Williamstown Portrait of George Washington.jpg'],
 'GND ID': ['11876439X'],
 'Library of Congress authority ID': ['n86140996'],
 'VIAF ID': ['31432428'],
 'ISNI': ['0000000120958599'],
 'IMDb ID': ['ch0028228'],
 'date of birth': ['+1732-